<a href="https://colab.research.google.com/github/daryoush/colab-julia-1.5/blob/master/startjulia1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### NOTE THIS WORKS AS PYTHON, IT ONLY WORKS WHEN THE RUNTIME COMPLAINS ABOUT JULIA RUNTIME, SO IT REVERTS TO PYTHON. AFTER JULIA IS INSTALLED, CHANGE TO JULIA RUNTIME

!curl -sSL "https://julialang-s3.julialang.org/bin/linux/x64/1.5/julia-1.5.0-linux-x86_64.tar.gz" -o julia.tar.gz
!tar -xzf julia.tar.gz -C /usr --strip-components 1
!rm -rf julia.tar.gz*
!julia -e 'using Pkg; pkg"add IJulia; precompile"'
!julia -e 'using Pkg; Pkg.add([ "CUDA", "Flux",\
    "Random","InformationMeasures", "DataStructures", "Transformers","Statistics", "Dates",\
    "BSON",\
    "Images",\
     "MLDatasets",\
    "Parameters",\
    "Printf",\
    ]);' ###Pkg.precompile()'
!echo "DONE"

 Installing known registries into `~/.julia`
######################################################################## 100.0%
      Added registry `General` to `~/.julia/registries/General`
  Resolving package versions...
  Installed ZMQ ───────────── v1.2.1
  Installed ZeroMQ_jll ────── v4.3.2+5
  Installed SoftGlobalScope ─ v1.1.0
  Installed Conda ─────────── v1.4.1
  Installed VersionParsing ── v1.2.0
  Installed IJulia ────────── v1.21.4
  Installed JSON ──────────── v0.21.1
  Installed JLLWrappers ───── v1.1.2
  Installed Artifacts ─────── v1.3.0
  Installed MbedTLS ───────── v1.0.2
  Installed Parsers ───────── v1.0.11
  Installed MbedTLS_jll ───── v2.16.8+0
######################################################################## 100.0%
######################################################################## 100.0%
Updating `~/.julia/environments/v1.5/Project.toml`
  [7073ff75] + IJulia v1.21.4
Updating `~/.julia/environments/v1.5/Manifest.toml`
  [56f22d72] + Artifacts v1.3.0
  

**NOW GO TO EDIT NOTEBOOK SETTING AND SET IT TO JULIA**
For adding new packages, it should be in python or the syntax may need to be in Julia!

In [1]:
[1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [ ]:
using CUDA
CuArray{Int}(undef, 2)

┌ Info: Precompiling CUDA [052768ef-5323-5732-b1bb-66c8b64840ba]
└ @ Base loading.jl:1278


######################################################################### 100.0%


######################################################################### 100.0%


######################################################################### 100.0%


In [1]:
using Flux

┌ Info: Precompiling Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1278


In [2]:
gpu([1,2,3])

3-element CUDA.CuArray{Int64,1}:
 1
 2
 3

In [1]:
##module minGPTonJulia


import Random
using Flux, Flux.Optimise
using Flux: crossentropy, Momentum
using Flux.Losses: logitcrossentropy
using Flux: onehotbatch, onecold, OneHotVector
using InformationMeasures
using DataStructures: SortedSet
using Transformers
using Statistics
using Transformers.Basic
using Flux: @functor
using BSON: @save, @load
using Dates

### We need to get a random value from the data, define it like distribution!!!
s = open("input.txt") do file
    read(file, String)
end
@show datasize=length(s)

chars=[SortedSet(s)...]
@show vocabSize=length(chars)

## each character gets the offset within the chars set.
itos = Dict(ic for ic in zip(1:vocabSize, chars))
stoi = Dict(ci for ci in zip(chars, 1:vocabSize))

encodeString(s) = [stoi[c] for c in s]
decodeOffsetToChar(i) = itos[i]
function randomData(noSamples, lengthOfSamples)
    res=[]
    for i in 1:noSamples
        ## For data size of 10 and 2 byte sample.
        ## get a random number from 0 to 10-2+1 (9)
        ## at extremes start = 1   get 1:1+2-1
        ###            start = 9   get 9:9+2-1
        start=rand(1:datasize-lengthOfSamples+1)   ## 0 to size-block length
        push!(res, s[start:start+lengthOfSamples-1])
    end
    res
end



###end # module

function trainingBatch(batchsize=511, blocksize=128)
    samples = hcat( encodeString.(randomData(batchsize, blocksize+1))...)
    xs=samples[1:end-1,:]
    ys=samples[2:end,:]
    xs,ys
end


# """
# GPT model:
# - the initial stem consists of a combination of token encoding and a positional encoding
# - the meat of it is a uniform sequence of Transformer blocks
#     - each Transformer is a sequential combination of a 1-hidden-layer MLP block and a self-attention block
#     - all blocks feed into a central residual pathway similar to resnets
# - the final decoder is a linear projection into a vanilla Softmax classifier
# """

struct minGPT{ EM<:Embed,
                PM<:PositionEmbedding,
				DP<:Dropout,
                C<:Chain,
				PW<:Positionwise
                }
    em::EM
    pe::PM
    emDrop::DP
	blocks::C
    # mh::MA
    # mhn::LA
    # pw::P
    # pwn::LP
    # drop::DP
	pw::PW
end

@functor minGPT

function minGPT(conf)
  @show conf["num_attention_layers"]
	attentions = ( Transformer(conf["hidden_size"],
							conf["num_attention_heads"],
							## hs is not specified, so internat attnetion size is calculated
							## ps is the size in the mlp block, 4 times the attention size
							conf["hidden_size"]*4;
							future=true,
							act=conf["hidden_act"],
							pdrop=conf["attn_pdrop"])
			for i in 1:conf["num_attention_layers"])
    minGPT(
			Embed(conf["hidden_size"], conf["vocab_size"]),
			PositionEmbedding(
	  			conf["hidden_size"],
	  			conf["max_position_embeddings"];
	  			trainable = true),
        	Dropout(conf["embd_pdrop"]),
			Chain(attentions...),

			#  NOTE THAT the multihead attention is not exported, but the transfomers are
			#  The transformer has a layer norm at the end
			#  for GPT we need to change the Pointwise and add gelu
			Positionwise(Dense(conf["hidden_size"], conf["vocab_size"])) #, logsoftmax)
    )
end

function (g::minGPT)(x::T) where T
	we = g.em(x, inv(sqrt(size(g.em.embedding)[1]))) ## use the sqrt of embeding size
	e = we .+ g.pe(we)
    drop=g.emDrop(e)
	dec=g.blocks(drop)
	pw=g.pw(dec)
end


const config = Dict(
  "attention_probs_dropout_prob" => 0.1,
  "initializer_range"            => 0.02,
  "hidden_act"                   => gelu,
  "num_hidden_layers"            => 12,
  "hidden_size"                  => 512,   ## embedding of each char in sentence
  "max_position_embeddings"      => 128,   ## length of sentences
  "type_vocab_size"              => 2,
  "vocab_size"                   => vocabSize,
  "num_attention_heads"          => 8,
  "num_attention_layers"         => 1, #12,
  "hidden_dropout_prob"          => 0.1,
  "intermediate_size"            => 3072,
  "embd_pdrop"					 => 0.1,
  "resid_pdrop" 			     => 0.1,
  "attn_pdrop"					 => 0.1
)



m = minGPT(config) |> gpu


## x will be the input string, y would be the predicted string from x
## y starts with 2nd character of x, and ends with character after the
## last
##  x and y would be the batch input and output
## Note the final value is agg over the batch, default agg is mean
function loss(x, y)
	desiredLastCharPostion = y[end,:]
	desiredLastCharOnehotEncoded = hcat([ OneHotVector(p, vocabSize)   for p in desiredLastCharPostion]...)
	predictedDistOfLastChar = m(x)[:,end,:]
	logitcrossentropy(predictedDistOfLastChar,desiredLastCharOnehotEncoded)
end


accuracy(x, y) = mean( onecold(softmax(m(x)[:,end,:]), chars) .== decodeOffsetToChar.(y[end,:]))


## Try a run of the network with random data
# note strings are encoded.  So the value of char is offset within the
# chars array.    So the last character of the desiredOutput is the
# offset of the deisred character.  That output of the model is distribution
# over the chars.   SO we do the cross entropy with the prob at the location
# of the desiredOutput's last char

const trainConfig = Dict(
					  "max_epochs" => 10,   # 200,
                      "batch_size" => 520,
					  "learning_rate" => 6e-4,
                      "lr_decay" => true,
					  "warmup_tokens" => 512*20,
                      "num_workers" => 4)

ps = params(m)
print(m)
opt = ADAM(trainConfig["learning_rate"])


modelSaveName = "model-$(now()).bson"
modelWeightName =  "model-weights$(now()).bson"

function train!()
	@info trainConfig["batch_size"], datasize÷trainConfig["batch_size"]
	@info "start training"
	for epoch = 1:trainConfig["max_epochs"]
	  @show epoch
	  for i = 1:datasize÷trainConfig["batch_size"]   ## sample random blocks
      d = trainingBatch(trainConfig["batch_size"]) |> gpu
		  @show ep,b, l = epoch, i, loss(d...)
		  grad = gradient(()->l, ps)
		  update!(opt, ps, grad)
		  if i % 10 == 0
			  @save  modelSaveName m
			  weights = params(m)
			  @save modelWeightName weights
		  end
	  end
	  @show accuracy(trainingBatch(trainConfig["batch_size"])...)

	end
end







datasize = length(s) = 1115394
vocabSize = length(chars) = 65
conf["num_attention_layers"] = 1
minGPT{Embed{Float32,CUDA.CuArray{Float32,2}},PositionEmbedding{Float32,CUDA.CuArray{Float32,2}},Dropout{Float64,Colon},Chain{Tuple{Transformer{Transformers.Basic.MultiheadAttention{Dense{typeof(identity),CUDA.CuArray{Float32,2},CUDA.CuArray{Float32,1}},Dense{typeof(identity),CUDA.CuArray{Float32,2},CUDA.CuArray{Float32,1}},Dense{typeof(identity),CUDA.CuArray{Float32,2},CUDA.CuArray{Float32,1}},Dense{typeof(identity),CUDA.CuArray{Float32,2},CUDA.CuArray{Float32,1}},Dropout{Float64,Colon}},LayerNorm{CUDA.CuArray{Float32,1}},Transformers.Basic.PwFFN{Dense{typeof(gelu),CUDA.CuArray{Float32,2},CUDA.CuArray{Float32,1}},Dense{typeof(identity),CUDA.CuArray{Float32,2},CUDA.CuArray{Float32,1}}},LayerNorm{CUDA.CuArray{Float32,1}},Dropout{Float64,Colon}}}},Positionwise{Tuple{Dense{typeof(identity),CUDA.CuArray{Float32,2},CUDA.CuArray{Float32,1}}}}}(Embed(512), PositionEmbedding(512, max_len=128), Dropou

train! (generic function with 1 method)

In [6]:
x,y  = trainingBatch(trainConfig["batch_size"])

([45 48 … 44 45; 54 53 … 51 2; … ; 31 33 … 58 47; 28 47 … 2 44], [54 53 … 51 2; 57 2 … 43 51; … ; 28 47 … 2 44; 24 48 … 59 2])

In [13]:
m2 = minGPT(config)   ## RAM VERSION

conf["num_attention_layers"] = 1


minGPT{Embed{Float32,Array{Float32,2}},PositionEmbedding{Float32,Array{Float32,2}},Dropout{Float64,Colon},Chain{Tuple{Transformer{Transformers.Basic.MultiheadAttention{Dense{typeof(identity),Array{Float32,2},Array{Float32,1}},Dense{typeof(identity),Array{Float32,2},Array{Float32,1}},Dense{typeof(identity),Array{Float32,2},Array{Float32,1}},Dense{typeof(identity),Array{Float32,2},Array{Float32,1}},Dropout{Float64,Colon}},LayerNorm{Array{Float32,1}},Transformers.Basic.PwFFN{Dense{typeof(gelu),Array{Float32,2},Array{Float32,1}},Dense{typeof(identity),Array{Float32,2},Array{Float32,1}}},LayerNorm{Array{Float32,1}},Dropout{Float64,Colon}}}},Positionwise{Tuple{Dense{typeof(identity),Array{Float32,2},Array{Float32,1}}}}}(Embed(512), PositionEmbedding(512, max_len=128), Dropout(0.1), Chain(Transformer(head=8, head_size=64, pwffn_size=2048, size=512)), Positionwise(Dense(512, 65)))

In [17]:
typeof(m2(x))

Array{Float32,3}

In [19]:
typeof(y)

Array{Int64,2}

In [26]:
function loss2(x, y)
	desiredLastCharPostion = y[end,:]
	desiredLastCharOnehotEncoded = hcat([ OneHotVector(p, vocabSize)   for p in desiredLastCharPostion]...)
	predictedDistOfLastChar = m2(x)[:,end,:]
	logitcrossentropy(predictedDistOfLastChar,desiredLastCharOnehotEncoded)
end

loss2 (generic function with 1 method)

In [29]:
loss2(x,y)

typeof(desiredLastCharPostion) = Array{Int64,1}
typeof(desiredLastCharOnehotEncoded) = Flux.OneHotMatrix{Array{OneHotVector,1}}
typeof(predictedDistOfLastChar) = Array{Float32,2}


4.935269f0

In [30]:
xg = x |> gpu
yg = y |> gpu
loss(xg, yg)

LoadError: ignored

In [22]:
typeof(chars)

Array{Char,1}

In [2]:
train!()

epoch = 1


┌ Info: (520, 2144)
└ @ Main In[1]:186
┌ Info: start training
└ @ Main In[1]:187
┌ Warning: Performing scalar operations on GPU arrays: This is very slow, consider disallowing these operations with `allowscalar(false)`
└ @ GPUArrays /root/.julia/packages/GPUArrays/uaFZh/src/host/indexing.jl:43


LoadError: ignored

In [ ]:

N = 2^10
x = fill(1.0f0, N)  # a vector filled with 1.0 (Float32)
y = fill(2.0f0, N)  # a vector filled with 2.0

x_d = CUDA.fill(1.0f0, N)  # a vector stored on the GPU filled with 1.0 (Float32)
y_d = CUDA.fill(2.0f0, N)  # a vector stored on the GPU filled with 2.0

1024-element CuArray{Float32,1}:
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 ⋮
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0

In [ ]:
@time  y .+= x

  0.000016 seconds (2 allocations: 64 bytes)


1024-element Array{Float32,1}:
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 ⋮
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0

In [ ]:
@time y_d .+= x_d

  0.000082 seconds (27 allocations: 704 bytes)


1024-element CuArray{Float32,1}:
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 ⋮
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0
 5.0

In [ ]:
# Classifies MNIST digits with a convolutional network.
# Writes out saved model to the file "mnist_conv.bson".
# Demonstrates basic model construction, training, saving,
# conditional early-exit, and learning rate scheduling.
#
# This model, while simple, should hit around 99% test
# accuracy after training for approximately 20 epochs.

using Flux, Flux.Data.MNIST, Statistics
using Flux: onehotbatch, onecold, logitcrossentropy
using Base.Iterators: partition
using Printf, BSON
using Parameters: @with_kw
#using CUDAapi
# if has_cuda()
#     @info "CUDA is on"
#     import CuArrays
#     CuArrays.allowscalar(false)
# end

@with_kw mutable struct Args
    lr::Float64 = 3e-3
    epochs::Int = 20
    batch_size = 128
    savepath::String = "./" 
end

# Bundle images together with labels and group into minibatchess
function make_minibatch(X, Y, idxs)
    X_batch = Array{Float32}(undef, size(X[1])..., 1, length(idxs))
    for i in 1:length(idxs)
        X_batch[:, :, :, i] = Float32.(X[idxs[i]])
    end
    Y_batch = onehotbatch(Y[idxs], 0:9)
    return (X_batch, Y_batch)
end

function get_processed_data(args)
    # Load labels and images from Flux.Data.MNIST
    train_labels = MNIST.labels()
    train_imgs = MNIST.images()
    mb_idxs = partition(1:length(train_imgs), args.batch_size)
    train_set = [make_minibatch(train_imgs, train_labels, i) for i in mb_idxs] 
    
    # Prepare test set as one giant minibatch:
    test_imgs = MNIST.images(:test)
    test_labels = MNIST.labels(:test)
    test_set = make_minibatch(test_imgs, test_labels, 1:length(test_imgs))

    return train_set, test_set

end

# Build model
function build_model(args; imgsize = (28,28,1), nclasses = 10)
    cnn_output_size = Int.(floor.([imgsize[1]/8,imgsize[2]/8,32]))	

    return Chain(
    # First convolution, operating upon a 28x28 image
    Conv((3, 3), imgsize[3]=>16, pad=(1,1), relu),
    MaxPool((2,2)),

    # Second convolution, operating upon a 14x14 image
    Conv((3, 3), 16=>32, pad=(1,1), relu),
    MaxPool((2,2)),

    # Third convolution, operating upon a 7x7 image
    Conv((3, 3), 32=>32, pad=(1,1), relu),
    MaxPool((2,2)),

    # Reshape 3d tensor into a 2d one using `Flux.flatten`, at this point it should be (3, 3, 32, N)
    flatten,
    Dense(prod(cnn_output_size), 10))
end

# We augment `x` a little bit here, adding in random noise. 
augment(x) = x .+ gpu(0.1f0*randn(eltype(x), size(x)))

# Returns a vector of all parameters used in model
paramvec(m) = vcat(map(p->reshape(p, :), params(m))...)

# Function to check if any element is NaN or not
anynan(x) = any(isnan.(x))

accuracy(x, y, model) = mean(onecold(cpu(model(x))) .== onecold(cpu(y)))

function train(; kws...)	
    args = Args(; kws...)

    @info("Loading data set")
    train_set, test_set = get_processed_data(args)

    # Define our model.  We will use a simple convolutional architecture with
    # three iterations of Conv -> ReLU -> MaxPool, followed by a final Dense layer.
    @info("Building model...")
    model = build_model(args) 

    # Load model and datasets onto GPU, if enabled
    train_set = gpu.(train_set)
    test_set = gpu.(test_set)
    model = gpu(model)
    
    # Make sure our model is nicely precompiled before starting our training loop
    model(train_set[1][1])

    # `loss()` calculates the crossentropy loss between our prediction `y_hat`
    # (calculated from `model(x)`) and the ground truth `y`.  We augment the data
    # a bit, adding gaussian random noise to our image to make it more robust.
    function loss(x, y)    
        x̂ = augment(x)
        ŷ = model(x̂)
        return logitcrossentropy(ŷ, y)
    end
	
    # Train our model with the given training set using the ADAM optimizer and
    # printing out performance against the test set as we go.
    opt = ADAM(args.lr)
	
    @info("Beginning training loop...")
    best_acc = 0.0
    last_improvement = 0
    for epoch_idx in 1:args.epochs
        # Train for a single epoch
        Flux.train!(loss, params(model), train_set, opt)
	    
        # Terminate on NaN
        if anynan(paramvec(model))
            @error "NaN params"
            break
        end
	
        # Calculate accuracy:
        acc = accuracy(test_set..., model)
		
        @info(@sprintf("[%d]: Test accuracy: %.4f", epoch_idx, acc))
        # If our accuracy is good enough, quit out.
        if acc >= 0.999
            @info(" -> Early-exiting: We reached our target accuracy of 99.9%")
            break
        end
	
        # If this is the best accuracy we've seen so far, save the model out
        if acc >= best_acc
            @info(" -> New best accuracy! Saving model out to mnist_conv.bson")
            BSON.@save joinpath(args.savepath, "mnist_conv.bson") params=cpu.(params(model)) epoch_idx acc
            best_acc = acc
            last_improvement = epoch_idx
        end
	
        # If we haven't seen improvement in 5 epochs, drop our learning rate:
        if epoch_idx - last_improvement >= 5 && opt.eta > 1e-6
            opt.eta /= 10.0
            @warn(" -> Haven't improved in a while, dropping learning rate to $(opt.eta)!")
   
            # After dropping learning rate, give it a few epochs to improve
            last_improvement = epoch_idx
        end
	
        if epoch_idx - last_improvement >= 10
            @warn(" -> We're calling this converged.")
            break
        end
    end
end

# Testing the model, from saved model
function test(; kws...)
    args = Args(; kws...)
    
    # Loading the test data
    _,test_set = get_processed_data(args)
    
    # Re-constructing the model with random initial weights
    model = build_model(args)
    
    # Loading the saved parameters
    BSON.@load joinpath(args.savepath, "mnist_conv.bson") params
    
    # Loading parameters onto the model
    Flux.loadparams!(model, params)
    
    test_set = gpu.(test_set)
    model = gpu(model)
    @show accuracy(test_set...,model)
end

cd(@__DIR__) 
train()
test()

┌ Info: Loading data set
└ @ Main In[13]:90
┌ Info: Building model...
└ @ Main In[13]:95
┌ Info: Beginning training loop...
└ @ Main In[13]:119
┌ Info: [1]: Test accuracy: 0.9820
└ @ Main In[13]:135
┌ Info:  -> New best accuracy! Saving model out to mnist_conv.bson
└ @ Main In[13]:144
┌ Info: [2]: Test accuracy: 0.9871
└ @ Main In[13]:135
┌ Info:  -> New best accuracy! Saving model out to mnist_conv.bson
└ @ Main In[13]:144
┌ Info: [3]: Test accuracy: 0.9888
└ @ Main In[13]:135
┌ Info:  -> New best accuracy! Saving model out to mnist_conv.bson
└ @ Main In[13]:144
┌ Info: [4]: Test accuracy: 0.9852
└ @ Main In[13]:135
┌ Info: [5]: Test accuracy: 0.9820
└ @ Main In[13]:135
┌ Info: [6]: Test accuracy: 0.9885
└ @ Main In[13]:135
┌ Info: [7]: Test accuracy: 0.9873
└ @ Main In[13]:135
┌ Info: [8]: Test accuracy: 0.9886
└ @ Main In[13]:135
┌ Warning:  -> Haven't improved in a while, dropping learning rate to 0.00030000000000000003!
└ @ Main In[13]:153
┌ Info: [9]: Test accuracy: 0.9926
└ @ Ma

accuracy(test_set..., model) = 0.9928


0.9928

In [ ]:
[1,2,3]

3-element Array{Int64,1}:
 1
 2
 3